In [7]:
import numpy as np
import pandas as pd
import requests
import matplotlib.pyplot as plt
from datetime import datetime
from tqdm import tqdm
import time
import os

In [28]:
df = pd.read_csv("datasets/all_nyc_crashes.csv")

essential_columns = [
    'CRASH DATE', 'CRASH TIME', 'LATITUDE', 'LONGITUDE', 'BOROUGH',
    'NUMBER OF PERSONS INJURED', 'NUMBER OF PERSONS KILLED',
    'NUMBER OF PEDESTRIANS INJURED', 'NUMBER OF PEDESTRIANS KILLED',
    'NUMBER OF CYCLIST INJURED', 'NUMBER OF CYCLIST KILLED',
    'NUMBER OF MOTORIST INJURED', 'NUMBER OF MOTORIST KILLED',
    'COLLISION_ID'
]

df = df[essential_columns]

/var/folders/xb/31y8q6395sb38dpcp8zcy7gc0000gn/T/ipykernel_83644/33070745.py:1: DtypeWarning: Columns (3) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv("datasets/all_nyc_crashes.csv")


In [29]:
df.dropna(inplace=True)
df.head(5)

,CRASH DATE,CRASH TIME,LATITUDE,LONGITUDE,BOROUGH,NUMBER OF PERSONS INJURED,NUMBER OF PERSONS KILLED,NUMBER OF PEDESTRIANS INJURED,NUMBER OF PEDESTRIANS KILLED,NUMBER OF CYCLIST INJURED,NUMBER OF CYCLIST KILLED,NUMBER OF MOTORIST INJURED,NUMBER OF MOTORIST KILLED,COLLISION_ID
2,11/01/2023,1:29,40.621790,-73.970024,BROOKLYN,1.0,0.0,0,0,0,0,1,0,4675373
9,09/11/2021,9:35,40.667202,-73.866500,BROOKLYN,0.0,0.0,0,0,0,0,0,0,4456314
10,12/14/2021,8:13,40.683304,-73.917274,BROOKLYN,0.0,0.0,0,0,0,0,0,0,4486609
13,12/14/2021,8:17,40.868160,-73.831480,BRONX,2.0,0.0,0,0,0,0,2,0,4486660
14,12/14/2021,21:10,40.671720,-73.897100,BROOKLYN,0.0,0.0,0,0,0,0,0,0,4487074


In [10]:
df.shape

(1498854, 14)

In [11]:
df['datetime'] = pd.to_datetime(
    df['CRASH DATE'] + ' ' + df['CRASH TIME'],
    format='%m/%d/%Y %H:%M'
)

df['iso_datetime'] = df['datetime'].dt.strftime('%Y-%m-%dT%H:%M')

df['year'] = df['datetime'].dt.year
df['month'] = df['datetime'].dt.month

In [12]:
print(df[['CRASH DATE', 'CRASH TIME', 'datetime']].head())

    CRASH DATE CRASH TIME            datetime
2   11/01/2023       1:29 2023-11-01 01:29:00
9   09/11/2021       9:35 2021-09-11 09:35:00
10  12/14/2021       8:13 2021-12-14 08:13:00
13  12/14/2021       8:17 2021-12-14 08:17:00
14  12/14/2021      21:10 2021-12-14 21:10:00


In [ ]:

def get_weather_data(lat, lon, date_time, timezone='America/New_York', retries=3, timeout=5):
    
    base_url = "https://archive-api.open-meteo.com/v1/archive"
    date_str = date_time[:10]  # YYYY-MM-DD
    hour_str = date_time[11:]  # HH:MM

    params = {
        'latitude': lat,
        'longitude': lon,
        'start_date': date_str,
        'end_date': date_str,
        'hourly': 'temperature_2m,precipitation,weathercode',
        'timezone': timezone,
    }

    for attempt in range(retries):
        try:
            response = requests.get(base_url, params=params, timeout=timeout)
            response.raise_for_status()
            data = response.json()

            # extracts weather for the specific hour
            target_time = f"{date_str}T{hour_str}"
            if 'hourly' in data and 'time' in data['hourly']:
                times = data['hourly']['time']
                if target_time in times:
                    index = times.index(target_time)
                    return {
                        'temperature': data['hourly']['temperature_2m'][index],
                        'precipitation': data['hourly']['precipitation'][index],
                        'weathercode': data['hourly']['weathercode'][index],
                    }
            return None
        except requests.exceptions.Timeout:
            print(f"Timeout occurred for lat: {lat}, lon: {lon}, datetime: {date_time} (attempt {attempt + 1}/{retries})")
        except requests.exceptions.HTTPError as e:
            print(f"HTTP error for lat: {lat}, lon: {lon}, datetime: {date_time}: {e}")
            break
        except requests.exceptions.RequestException as e:
            print(f"Request error for lat: {lat}, lon: {lon}, datetime: {date_time}: {e}")
        time.sleep(1)

    print(f"Skipping lat: {lat}, lon: {lon}, datetime: {date_time} after {retries} attempts.")
    return None


In [14]:
def map_weathercode_to_description(weathercode):
    if weathercode in [0, 1]:
        return 'Clear'
    elif weathercode in [2, 3]:
        return 'Cloudy'
    elif weathercode in [45, 48]:
        return 'Fog/Haze'
    elif weathercode in [51, 53, 55, 56, 57]:
        return 'Drizzle'
    elif weathercode in [61, 80]:
        return 'Light Rain'
    elif weathercode in [63, 65, 66, 67, 81, 82]:
        return 'Heavy Rain'
    elif weathercode in [71, 85]:
        return 'Light Snow'
    elif weathercode in [73, 75, 77, 86]:
        return 'Heavy Snow'
    elif weathercode in [95, 96, 99]:
        return 'Thunderstorm'
    else:
        return 'Unknown'


In [15]:
def build_yearly_sample(
    base_df,
    year,
    non_severe_frac=0.10,
    max_per_month=1000,
    max_per_borough=2000,
    random_state=42
):
    """
    Creates a sampled DataFrame for a given year:
      - includes ALL severe crashes (injured or killed)
      - includes a random fraction of non-severe crashes
      - caps per month (seasons) and per borough (areas)
      - deduplicates on (LATITUDE, LONGITUDE, datetime)
    """
    year_df = base_df[base_df['year'] == year].copy()
    if year_df.empty:
        print(f"No records for year {year}.")
        return pd.DataFrame()

    severe_mask = (year_df['NUMBER OF PERSONS INJURED'] > 0) | (year_df['NUMBER OF PERSONS KILLED'] > 0)
    severe_accidents = year_df[severe_mask]
    non_severe = year_df[~severe_mask]

    if not non_severe.empty:
        non_severe_sample = non_severe.sample(
            frac=min(non_severe_frac, 1.0),
            random_state=random_state
        )
    else:
        non_severe_sample = pd.DataFrame(columns=year_df.columns)

    sampled_df = pd.concat([severe_accidents, non_severe_sample])

    # Balance across months (seasons)
    sampled_df = (
        sampled_df
        .groupby('month', group_keys=False)
        .apply(lambda x: x.sample(min(len(x), max_per_month), random_state=random_state))
    )

    # Balance across boroughs (areas)
    sampled_df = (
        sampled_df
        .groupby('BOROUGH', group_keys=False)
        .apply(lambda x: x.sample(min(len(x), max_per_borough), random_state=random_state))
    )

    sampled_df = sampled_df.drop_duplicates(subset=['LATITUDE', 'LONGITUDE', 'datetime'])

    print(f"Year {year}: sampled {sampled_df.shape[0]} rows.")
    return sampled_df


In [16]:
def enrich_with_weather(sample_df):
    if sample_df.empty:
        return sample_df

    sample_df = sample_df.copy()
    sample_df['temperature'] = np.nan
    sample_df['precipitation'] = np.nan
    sample_df['weather_condition'] = np.nan

    for index, row in tqdm(sample_df.iterrows(), total=sample_df.shape[0]):
        weather_data = get_weather_data(row['LATITUDE'], row['LONGITUDE'], row['iso_datetime'])
        if weather_data:
            sample_df.at[index, 'temperature'] = weather_data['temperature']
            sample_df.at[index, 'precipitation'] = weather_data['precipitation']
            sample_df.at[index, 'weather_condition'] = map_weathercode_to_description(weather_data['weathercode'])

    sample_df.dropna(subset=['temperature', 'precipitation', 'weather_condition'], inplace=True)

    return sample_df

In [17]:
def sample_specific_years(
    base_df,
    years=range(2016, 2026),
    non_severe_frac=0.10,
    max_per_month=1000,
    max_per_borough=2000,
    random_state=42
):

    year_to_df = {}

    for y in years:
        if y not in base_df["year"].unique():
            print(f"⚠️ Year {y} not found in dataset, skipping.")
            continue

        print(f"\n📌 Processing year: {y}")
        sampled_y = build_yearly_sample(
            base_df,
            year=y,
            non_severe_frac=non_severe_frac,
            max_per_month=max_per_month,
            max_per_borough=max_per_borough,
            random_state=random_state
        )

        if sampled_y.empty:
            print(f"⚠️ No rows sampled for year {y}.")
            continue

        year_to_df[y] = sampled_y

    return year_to_df


In [18]:
yearly_samples = sample_specific_years(df)

sample_2016 = yearly_samples.get(2016)
sample_2017 = yearly_samples.get(2017)
sample_2018 = yearly_samples.get(2018)
sample_2019 = yearly_samples.get(2019)
sample_2020 = yearly_samples.get(2020)
sample_2021 = yearly_samples.get(2021)
sample_2022 = yearly_samples.get(2022)
sample_2023 = yearly_samples.get(2023)
sample_2024 = yearly_samples.get(2024)
sample_2025 = yearly_samples.get(2025)



📌 Processing year: 2016
Year 2016: sampled 8254 rows.

📌 Processing year: 2017
Year 2017: sampled 8365 rows.

📌 Processing year: 2018
Year 2018: sampled 8395 rows.

📌 Processing year: 2019
Year 2019: sampled 8314 rows.

📌 Processing year: 2020
Year 2020: sampled 8134 rows.

📌 Processing year: 2021


/var/folders/xb/31y8q6395sb38dpcp8zcy7gc0000gn/T/ipykernel_83644/1311205039.py:37: FutureWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  sampled_df
/var/folders/xb/31y8q6395sb38dpcp8zcy7gc0000gn/T/ipykernel_83644/1311205039.py:44: FutureWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  sampled_df
/var/folders/xb/31y8q6395sb38dpcp8zcy7gc0000gn/T/ipykernel_83644/1311205039.py:37: FutureWarning: DataFrameGroupBy.apply operated on the gro

Year 2021: sampled 8257 rows.

📌 Processing year: 2022
Year 2022: sampled 8443 rows.

📌 Processing year: 2023
Year 2023: sampled 8309 rows.

📌 Processing year: 2024
Year 2024: sampled 8354 rows.

📌 Processing year: 2025
Year 2025: sampled 7754 rows.


/var/folders/xb/31y8q6395sb38dpcp8zcy7gc0000gn/T/ipykernel_83644/1311205039.py:37: FutureWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  sampled_df
/var/folders/xb/31y8q6395sb38dpcp8zcy7gc0000gn/T/ipykernel_83644/1311205039.py:44: FutureWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  sampled_df
/var/folders/xb/31y8q6395sb38dpcp8zcy7gc0000gn/T/ipykernel_83644/1311205039.py:37: FutureWarning: DataFrameGroupBy.apply operated on the gro

In [ ]:
def process_single_year(base_df, year, output_dir="datasets"):
    os.makedirs(output_dir, exist_ok=True)

    print(f"Sampling {year}...")
    sampled_df = build_yearly_sample(base_df, year)

    print(f"Enriching {year}...")
    enriched_df = enrich_with_weather(sampled_df)

    output_path = f"{output_dir}/enriched_{year}.csv"
    enriched_df.to_csv(output_path, index=False)

    print(f"Saved enriched dataset for {year} to {output_path}")
    return enriched_df

In [27]:
process_single_year(df, 2024)

/var/folders/xb/31y8q6395sb38dpcp8zcy7gc0000gn/T/ipykernel_83644/1311205039.py:37: FutureWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  sampled_df
/var/folders/xb/31y8q6395sb38dpcp8zcy7gc0000gn/T/ipykernel_83644/1311205039.py:44: FutureWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  sampled_df


Sampling 2024...
Year 2024: sampled 8354 rows.
Enriching 2024...


  0%|          | 0/8354 [00:00<?, ?it/s]/var/folders/xb/31y8q6395sb38dpcp8zcy7gc0000gn/T/ipykernel_83644/3211067085.py:15: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value 'Cloudy' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  sample_df.at[index, 'weather_condition'] = map_weathercode_to_description(weather_data['weathercode'])
 27%|██▋       | 2266/8354 [23:21<1:02:46,  1.62it/s]

Timeout occurred for lat: 40.68093, lon: -74.004944, datetime: 2024-07-08T16:45 (attempt 1/3)
Timeout occurred for lat: 40.68093, lon: -74.004944, datetime: 2024-07-08T16:45 (attempt 2/3)
Timeout occurred for lat: 40.68093, lon: -74.004944, datetime: 2024-07-08T16:45 (attempt 3/3)


 27%|██▋       | 2267/8354 [23:45<12:36:59,  7.46s/it]

...

 96%|█████████▌| 8031/8354 [1:24:46<03:18,  1.62it/s]

HTTP error for lat: 40.56264, lon: -74.19259, datetime: 2024-12-19T00:12: 429 Client Error: Too Many Requests for url: https://archive-api.open-meteo.com/v1/archive?latitude=40.56264&longitude=-74.19259&start_date=2024-12-19&end_date=2024-12-19&hourly=temperature_2m%2Cprecipitation%2Cweathercode&timezone=America%2FNew_York
Skipping lat: 40.56264, lon: -74.19259, datetime: 2024-12-19T00:12 after 3 attempts.


100%|██████████| 8354/8354 [1:28:04<00:00,  1.58it/s]

Saved enriched dataset for 2024 to outputs/enriched_2024.csv


,CRASH DATE,CRASH TIME,LATITUDE,LONGITUDE,BOROUGH,NUMBER OF PERSONS INJURED,NUMBER OF PERSONS KILLED,NUMBER OF PEDESTRIANS INJURED,NUMBER OF PEDESTRIANS KILLED,NUMBER OF CYCLIST INJURED,...,NUMBER OF MOTORIST INJURED,NUMBER OF MOTORIST KILLED,COLLISION_ID,datetime,iso_datetime,year,month,temperature,precipitation,weather_condition
310031,03/17/2024,1:00,40.862755,-73.901085,BRONX,1.0,0.0,0,0,0,...,1,0,4711491,2024-03-17 01:00:00,2024-03-17T01:00,2024,3,7.7,0.0,Cloudy
263559,02/12/2024,0:00,40.904243,-73.852165,BRONX,2.0,0.0,0,0,0,...,2,0,4702550,2024-02-12 00:00:00,2024-02-12T00:00,2024,2,3.3,0.0,Cloudy
308930,03/17/2024,10:00,40.866577,-73.867325,BRONX,1.0,0.0,1,0,0,...,0,0,4710755,2024-03-17 10:00:00,2024-03-17T10:00,2024,3,12.4,0.0,Clear
2116648,09/10/2024,0:00,40.876340,-73.863205,BRONX,1.0,0.0,0,0,0,...,1,0,4754452,2024-09-10 00:00:00,2024-09-10T00:00,2024,9,15.9,0.0,Clear
310411,05/11/2024,0:00,40.890076,-73.819855,BRONX,1.0,0.0,0,0,0,...,0,0,4727594,2024-05-11 00:00:00,2024-05-11T00:00,2024,5,9.0,0.0,Cloudy
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2132019,09/10/2024,18:00,40.578110,-74.111206,STATEN ISLAND,4.0,0.0,0,0,0,...,4,0,4754929,2024-09-10 18:00:00,2024-09-10T18:00,2024,9,23.6,0.0,Clear
289993,02/25/2024,8:00,40.524180,-74.234800,STATEN ISLAND,1.0,0.0,0,0,0,...,1,0,4705052,2024-02-25 08:00:00,2024-02-25T08:00,2024,2,-4.2,0.0,Clear
292617,05/13/2024,21:00,40.624603,-74.079605,STATEN ISLAND,2.0,0.0,0,0,0,...,2,0,4724762,2024-05-13 21:00:00,2024-05-13T21:00,2024,5,14.5,0.0,Clear
2123017,08/12/2024,15:00,40.610996,-74.095810,STATEN ISLAND,1.0,0.0,0,0,1,...,0,0,4747562,2024-08-12 15:00:00,2024-08-12T15:00,2024,8,26.1,0.2,Drizzle
